In [ ]:
# https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification

# https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a

In [ ]:
%%capture
!pip install -r /content/requirements.txt ;

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
data = pd.read_csv("all_translated_data.csv", encoding = "utf-8")
data["claim"] = data["majority_label"].apply(lambda x: 1 if x == "Yes" else (0 if x == "No" else "No_Majority"))
# Just use the texts with a majority label
data = data[data.claim != "No_Majority"]
data = data.reset_index(drop=True)
data.head()

,Unnamed: 0,text,language,majority_label,source,annotator1,annotator2,annotator3,translated_texts,claim
0,0,ഇച്ഛാശക്തിയുള്ള ഒരു ഭരണകൂടത്തിന് അസാധ്യമായൊന്ന...,ml,Yes,social media,Yes,Yes,Yes,There is nothing impossible for a government t...,1
1,1,. *_SKY ViSiON_* / / ...,ml,Yes,social media,Yes,Yes,Yes,"== sync, corrected by elderman == @elder_man =...",1
2,2,+------+------+-----+------+-----+-----+------...,ml,Yes,social media,No,Yes,Yes,The-------------------------------------------...,1
3,4,ബിജെപി അല്ലെങ്കിൽ യുപിയിലെ സഖ്യം... ഇവരിൽ നിന്...,ml,Yes,social media,Yes,No,Yes,Bijepi or the party in Jupiter... any of them ...,1
4,5,. *_SKY ViSiON_* / / ...,ml,Yes,social media,Yes,Yes,Yes,*_SKY VISION_* / /_size_**_bulance_*_bulance_*...,1


In [ ]:
# Select all languages
data_en= data.loc[(data.language == "en")]
data_hi = data.loc[(data.language == "hi")]
data_bn = data.loc[(data.language == "bn")]
data_ta = data.loc[(data.language == "ta")]
data_ml = data.loc[(data.language == "ml")]

In [ ]:
# Use this to make a bigger Dataframe with the selected language and the translated texts in the same language

text_hi = data_hi["text"].tolist()
text_trans_hi = data_hi["translated_texts"].tolist()
claim_hi = data_hi["claim"].tolist()
claim_trans_hi = data_hi["claim"].tolist()
text_list_hi = text_hi + text_trans_hi
claim_list_hi = claim_hi + claim_trans_hi
df_dict_hi = {"text": text_list_hi, "claim": claim_list_hi}
df_dict_hi
hi_trans_df = pd.DataFrame(df_dict_hi)

text_ta = data_ta["text"].tolist()
text_trans_ta = data_ta["translated_texts"].tolist()
claim_ta = data_ta["claim"].tolist()
claim_trans_ta = data_ta["claim"].tolist()
text_list_ta = text_ta + text_trans_ta
claim_list_ta = claim_ta + claim_trans_ta
df_dict_ta = {"text": text_list_ta, "claim": claim_list_ta}
df_dict_ta
ta_trans_df = pd.DataFrame(df_dict_ta)

text_ml = data_ml["text"].tolist()
text_trans_ml = data_ml["translated_texts"].tolist()
claim_ml = data_ml["claim"].tolist()
claim_trans_ml = data_ml["claim"].tolist()
text_list_ml = text_ml + text_trans_ml
claim_list_ml = claim_ml + claim_trans_ml
df_dict_ml = {"text": text_list_ml, "claim": claim_list_ml}
df_dict_ml
ml_trans_df = pd.DataFrame(df_dict_ml)

text_bn = data_bn["text"].tolist()
text_trans_bn = data_bn["translated_texts"].tolist()
claim_bn = data_bn["claim"].tolist()
claim_trans_bn = data_bn["claim"].tolist()
text_list_bn = text_bn + text_trans_bn
claim_list_bn = claim_bn + claim_trans_bn
df_dict_bn = {"text": text_list_bn, "claim": claim_list_bn}
df_dict_bn
bn_trans_df = pd.DataFrame(df_dict_bn)

In [ ]:
data_hi

,Unnamed: 0,text,language,majority_label,source,annotator1,annotator2,annotator3,translated_texts,claim
3145,4066,"*कुसुम किसलय कुञ्ज कोकिल,*\n*कूकते है फ़ाग में।...",hi,No,social media,No,No,No,"* Kuzum Kway Kwanj Kokil, * * Kukukuk in Fagh....",0
3146,4067,जिसने जेल में 'कसाब' कोे 'कबाब' दिया और 'साध्व...,hi,No,social media,No,No,No,Who gave 'Casab' to 'Casab' in prison and 'Saa...,0
3147,4068,मेरी गली में एक आवारा कुत्ता है। बेबजह भौंकता ...,hi,No,social media,No,No,No,There's an air dog in my neck. I don't know.,0
3148,4069,\nमौसम_अलर्ट:-\n\nसक्रिय प०वि० के प्रभाव से पा...,hi,Yes,social media,No,Yes,Yes,Moussam_Alert:- Active dog breeders are becomi...,1
3149,4070,शाहीन बाग में प्रदर्शन के दौरान महिलाओं के बीच...,hi,Yes,fact-check,Yes,Yes,Yes,Marpet between women during the show in Shaun'...,1
...,...,...,...,...,...,...,...,...,...,...
4012,5061,सर हम लोगो को उत्तर प्रदेश बिजली विभाग ने तीन ...,hi,No,social media,No,No,No,"Sir, we have brought out the people from the w...",0
4013,5062,_*आज के श्रृंगार दर्शन श्री गणेश जी के खजराना ...,hi,No,social media,No,No,Probably,_*Today's fair exhibition Mr. G. G.'s treasure...,0
4014,5063,Fact Check: लोकसभा चुनावों में वोट नहीं देने प...,hi,Yes,fact-check,Yes,Yes,Probably,Fact Check: If you don't vote in public electi...,1
4015,5064,Fact Check: फारूक अब्दुल्ला के “भारत माता की ज...,hi,Yes,fact-check,Yes,No,Yes,Fact Check: Faruk Abdulla's home is not after ...,1


In [ ]:
# Create Train, Test, Validate for every language
#data_en = data_en[["text", "claim"]]
data_hi = hi_trans_df[["text", "claim"]]
data_ml = ml_trans_df[["text", "claim"]]
data_ta = ta_trans_df[["text", "claim"]]
data_bn = bn_trans_df[["text", "claim"]]

data_en = data_en.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_hi = data_hi.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_ml = data_ml.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_ta = data_ta.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')
data_bn = data_bn.rename({'text': 'sentence1', 'claim': 'label'}, axis='columns')

#set seed
np.random.seed(0)
train_en, validate_en, test_en = np.split(data_en.sample(frac=1), [int(.6*len(data_en)), int(.8*len(data_en))])
train_en.to_csv("train_en.csv")
validate_en.to_csv("validate_en.csv")
test_en.to_csv("test_en.csv")

#set seed
np.random.seed(0)
train_hi, validate_hi, test_hi = np.split(data_hi.sample(frac=1), [int(.6*len(data_hi)), int(.8*len(data_hi))])
train_hi.to_csv("train_hi.csv")
validate_hi.to_csv("validate_hi.csv")
test_hi.to_csv("test_hi.csv")

#set seed
np.random.seed(0)
train_ml, validate_ml, test_ml = np.split(data_ml.sample(frac=1), [int(.6*len(data_ml)), int(.8*len(data_ml))])
train_ml.to_csv("train_ml.csv")
validate_ml.to_csv("validate_ml.csv")
test_ml.to_csv("test_ml.csv")

#set seed
np.random.seed(0)
train_ta, validate_ta, test_ta = np.split(data_ta.sample(frac=1), [int(.6*len(data_ta)), int(.8*len(data_ta))])
train_ta.to_csv("train_ta.csv")
validate_ta.to_csv("validate_ta.csv")
test_ta.to_csv("test_ta.csv")

#set seed
np.random.seed(0)
train_bn, validate_bn, test_bn = np.split(data_bn.sample(frac=1), [int(.6*len(data_bn)), int(.8*len(data_bn))])
train_bn.to_csv("train_bn.csv")
validate_bn.to_csv("validate_bn.csv")
test_bn.to_csv("test_bn.csv")

In [ ]:
print(len(train_ml))
print(len(test_ml))
print(len(validate_ml))

1164
388
388


In [ ]:
test_ml[:5]

,Unnamed: 0,sentence1,label
0,800,ബിജെപി ബിഡിജെഎസിനെ ചതിച്ചു...തുഷാറിന്‍റെ പ്രചാ...,0
1,281,നോമ്പ് മുസൽമാന്‌ നിർബന്ധം ഉള്ള കാര്യമാ എന്നാൽ ...,1
2,932,*തിരുവനന്തപുരം നഗരത്തിലെ എല്ലാ സര്‍ക്കാര്‍ സ്ഥ...,1
3,732,*(കേരളത്തിൽ രാഹുൽ ഗാന്ധി മത്സരിക്കുന്നതിനെ കുറ...,1
4,715,. *_SKY VISION_* / / ...,1


In [ ]:
!python text_class.py \
--model_name_or_path google/muril-base-cased \
--train_file train_en.csv \
--validation_file validate_en.csv \
--output_dir output_en/ \
--test_file test_en.csv

01/20/2022 09:14:55 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path google/muril-base-cased \
--train_file train_bn.csv \
--validation_file validate_bn.csv \
--output_dir output_bn/ \
--test_file test_bn.csv

01/20/2022 09:18:13 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path google/muril-base-cased \
--train_file train_hi.csv \
--validation_file validate_hi.csv \
--output_dir output_hi/ \
--test_file test_hi.csv

01/20/2022 09:21:56 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path google/muril-base-cased \
--train_file train_ml.csv \
--validation_file validate_ml.csv \
--output_dir output_ml/ \
--test_file test_ml.csv

01/20/2022 09:24:49 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
!python text_class.py \
--model_name_or_path google/muril-base-cased \
--train_file train_ta.csv \
--validation_file validate_ta.csv \
--output_dir output_ta/ \
--test_file test_ta.csv

01/20/2022 09:28:38 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
lo

In [ ]:
test_en_res = pd.read_csv('output_en/test_results.txt', sep="\t")
test_en = pd.read_csv('test_en.csv')
test_en_res = test_en_res.prediction
test_en.label

print("EN: Classification report: \n", (classification_report(test_en.label, test_en_res)))
print("EN: F1 micro averaging:",(f1_score(test_en.label, test_en_res, average='micro')))
print("EN: F1 macro averaging:",(f1_score(test_en.label, test_en_res, average='macro')))

EN: Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.11      0.19        84
           1       0.57      1.00      0.73       101

    accuracy                           0.59       185
   macro avg       0.79      0.55      0.46       185
weighted avg       0.77      0.59      0.49       185

EN: F1 micro averaging: 0.5945945945945946
EN: F1 macro averaging: 0.46139513217654593


In [ ]:
test_hi_res = pd.read_csv('output_hi/test_results.txt', sep="\t")
test_hi = pd.read_csv('test_hi.csv')
test_hi_res = test_hi_res.prediction
test_hi.label

print("HI: Classification report: \n", (classification_report(test_hi.label, test_hi_res)))
print("HI: F1 micro averaging:",(f1_score(test_hi.label, test_hi_res, average='micro')))
print("HI: F1 macro averaging:",(f1_score(test_hi.label, test_hi_res, average='macro')))

HI: Classification report: 
               precision    recall  f1-score   support

           0       0.57      0.98      0.72       173
           1       0.92      0.27      0.42       176

    accuracy                           0.62       349
   macro avg       0.75      0.62      0.57       349
weighted avg       0.75      0.62      0.57       349

HI: F1 micro averaging: 0.6217765042979942
HI: F1 macro averaging: 0.57010078387458


In [ ]:
test_ml_res = pd.read_csv('output_ml/test_results.txt', sep="\t")
test_ml = pd.read_csv('test_ml.csv')
test_ml_res = test_ml_res.prediction
test_ml.label

print("ML: Classification report: \n", (classification_report(test_ml.label, test_ml_res)))
print("ML: F1 micro averaging:",(f1_score(test_ml.label, test_ml_res, average='micro')))
print("ML: F1 macro averaging:",(f1_score(test_ml.label, test_ml_res, average='macro')))

ML: Classification report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       104
           1       0.73      1.00      0.85       284

    accuracy                           0.73       388
   macro avg       0.37      0.50      0.42       388
weighted avg       0.54      0.73      0.62       388

ML: F1 micro averaging: 0.7319587628865979
ML: F1 macro averaging: 0.42261904761904756


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_ta_res = pd.read_csv('output_ta/test_results.txt', sep="\t")
test_ta = pd.read_csv('test_ta.csv')
test_ta_res = test_ta_res.prediction
test_ta.label

print("TA: Classification report: \n", (classification_report(test_ta.label, test_ta_res)))
print("TA: F1 micro averaging:",(f1_score(test_ta.label, test_ta_res, average='micro')))
print("TA: F1 macro averaging:",(f1_score(test_ta.label, test_ta_res, average='macro')))

TA: Classification report: 
               precision    recall  f1-score   support

           0       0.87      0.87      0.87        45
           1       0.93      0.93      0.93        87

    accuracy                           0.91       132
   macro avg       0.90      0.90      0.90       132
weighted avg       0.91      0.91      0.91       132

TA: F1 micro averaging: 0.9090909090909091
TA: F1 macro averaging: 0.8988505747126436


In [ ]:
test_bn_res = pd.read_csv('output_bn/test_results.txt', sep="\t")
test_bn = pd.read_csv('test_bn.csv')
test_bn_res = test_bn_res.prediction
test_bn.label

print("BN: Classification report: \n", (classification_report(test_bn.label, test_bn_res)))
print("BN: F1 micro averaging:",(f1_score(test_bn.label, test_bn_res, average='micro')))
print("BN: F1 macro averaging:",(f1_score(test_bn.label, test_bn_res, average='macro')))

BN: Classification report: 
               precision    recall  f1-score   support

           0       0.76      0.96      0.85       241
           1       0.86      0.42      0.57       128

    accuracy                           0.78       369
   macro avg       0.81      0.69      0.71       369
weighted avg       0.79      0.78      0.75       369

BN: F1 micro averaging: 0.7750677506775068
BN: F1 macro averaging: 0.7068541401456779


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.makedirs('/content/drive/MyDrive/Multi_XLM/output_en')
os.makedirs('/content/drive/MyDrive/Multi_XLM/output_hi')
os.makedirs('/content/drive/MyDrive/Multi_XLM/output_ml')
os.makedirs('/content/drive/MyDrive/Multi_XLM/output_bn')
os.makedirs('/content/drive/MyDrive/Multi_XLM/output_ta')

In [ ]:
!cp output_en/* /content/drive/MyDrive/Multi_XLM/output_en
!cp output_hi/* /content/drive/MyDrive/Multi_XLM/output_hi
!cp output_ta/* /content/drive/MyDrive/Multi_XLM/output_ta
!cp output_ml/* /content/drive/MyDrive/Multi_XLM/output_ml
!cp output_bn/* /content/drive/MyDrive/Multi_XLM/output_bn

In [ ]:
!cp * /content/drive/MyDrive/Multi_XLM/

cp: -r not specified; omitting directory 'drive'
cp: -r not specified; omitting directory 'output_bn'
cp: -r not specified; omitting directory 'output_en'
cp: -r not specified; omitting directory 'output_hi'
cp: -r not specified; omitting directory 'output_ml'
cp: -r not specified; omitting directory 'output_ta'
cp: -r not specified; omitting directory 'sample_data'
